# Part I. ETL Pipeline:
### Pre-Process the Source CSV files and create a consolidated CSV file for loading Apache Cassandra tables.

### Source/Target:
    * Source(s) : Event files dataset (of CSV format)
    * Target(s) : Consolidated CSV file named as `event_datafile_new.csv`

#### Import Python packages 

In [32]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [33]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # Exclude the .ipynb_checkpoints directory if running thru Jupyter notebook
    dirs[:] = [d for d in dirs if d not in ['.ipynb_checkpoints']]
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [34]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [35]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print("Number of records in consolidated csv file = " + str(sum(1 for line in f)))

Number of records in consolidated csv file = 6821


# Part II. Creation, Modeling and Loading of Apache Cassandra tables. 
### Create 3 new Apache Cassandra tables to optimize the 3 user queries as follows:
    * Get the artist, song title and song's length in the music app history that was heard during a given session_id and item_in_session. (Eg: sessionId = 338, and itemInSession  = 4)
    
    * Get the artist, song (sorted by itemInSession) and user (first and last name) for a given user_id and session_id (For Eg: user_id = 10, session_id = 182) 

    * Get the user name (first and last) from the music app history who listened to a given song (For Eg: 'All Hands Against His Own') 

### Source/Target:
    * Source(s) : Consolidated CSV file named as `event_datafile_new.csv`
    * Target(s) : Apache Cassandra tables modelled for each user requirements 

#### The <font color=red>event_datafile_new.csv</font> created in the previous step contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

#### Creating a Apache Cassandra Cluster

In [36]:
# Connect to a Cassandra instance

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])

    # Create a session for the connected Cassandra instance
    session = cluster.connect()
    
except Exception as e:
    print(e)

#### Create Keyspace

In [37]:
# Create Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [38]:
# Set KEYSPACE to the sparkify keyspace which is created in previous step
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Requirement #1:

#### Model and Create the following table to get the artist, song title and song's length in the music app history that was heard during a given session_id and item_in_session. (Eg: sessionId = 338, and itemInSession  = 4)

To optimize the performance of above query, the below Cassandra table is created:

Table Name : `music_app_history_session_item`

Columns    : `session_id, item_in_session, artist, song, length`

#### Drop table `music_app_history_session_item` if already exists

In [39]:
query = "DROP TABLE IF EXISTS music_app_history_session_item"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Create table `music_app_history_session_item` based on the Requirement #1 mentioned above.

In [40]:
## Populate CREATE TABLE statment into query variable and execute it.
query = "CREATE TABLE IF NOT EXISTS music_app_history_session_item"
query = query + "(session_id int, item_in_session int, artist text, song text, length float, primary key(session_id, item_in_session))"  
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Load data into table `music_app_history_session_item`

In [41]:
file = 'event_datafile_new.csv'

# Open the csv file in read mode. 
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    # Loop thru each line and insert the data into Cassandra table music_app_history_session_item.
    for line in csvreader:
        ## Populate the INSERT statements into the `query` variable to insert data for session_id, item_in_session, artist, song, length
        query = "INSERT INTO music_app_history_session_item(session_id, item_in_session, artist, song, length)"
        query = query + "values(%s, %s, %s, %s, %s)"
        
        ## Execute the INSERT statement by passing the respective column values.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT on `music_app_history_session_item` table based on following conditions to verify the data.
 * Columns to be selected: artist, song title and song's length 
 * WHERE Condition : session_id = 338, item_in_session = 4

In [42]:
# Execute the SELECT statement
query = "select artist, song, length from music_app_history_session_item where session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Loop thru the SELECT query results and display the output in console
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Requirement #2:

#### Model and Create the following table to get the artist, song (sorted by itemInSession) and user (first and last name) for a given user_id and session_id (For Eg: user_id = 10, session_id = 182) 

To optimize the performance of above query, the below Cassandra table is created:

Table Name : `music_app_history_user_session`

Columns    : `user_id, session_id, item_in_session, artist, song, first_name, last_name`

#### Drop table `music_app_history_user_session`

In [43]:
query = "DROP TABLE IF EXISTS music_app_history_user_session"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Create table `music_app_history_user_session` based on the Requirement #2 mentioned above.

In [44]:
## Populate CREATE TABLE statment into query variable and execute it.
query = "CREATE TABLE IF NOT EXISTS music_app_history_user_session"
query = query + "(user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text, primary key((user_id, session_id), item_in_session))"  
try:
    session.execute(query)
except Exception as e:
    print(e)                    

#### Load data into table `music_app_history_user_session`

In [45]:
file = 'event_datafile_new.csv'
result=[]
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        ## Populate the INSERT statements into the `query` variable to insert data for user_id, session_id, item_in_session, artist, song, first_name, last_name
        query = "INSERT INTO music_app_history_user_session(user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + "values(%s, %s, %s, %s, %s, %s, %s)"
        ## Execute the INSERT statement by passing the respective column values.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Do a SELECT on `music_app_history_user_session` table based on following conditions to verify the data.
 * Columns to be selected: name of artist, song (sorted by itemInSession) and user (first and last name) 
 * WHERE Condition : user_id = 10, session_id = 182

In [46]:
# Execute the SELECT statement
query = "select artist, song, first_name, last_name from music_app_history_user_session where user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Loop thru the SELECT query results and display the output in console
for row in rows:
    print (row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Requirement #3:

#### Model and Create the following table to get the user name (first and last) from the music app history who listened to a given song (For Eg: 'All Hands Against His Own') 

To optimize the performance of above query, the below Cassandra table is created:

Table Name : `music_app_history_song`

Columns    : `song, user_id, first_name, last_name`

#### Drop table `music_app_history_song`

In [47]:
## Populate DROP TABLE statment into query variable and execute it.
query = "DROP TABLE IF EXISTS music_app_history_song"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Create the table `music_app_history_song` based on the Requirement #3 mentioned above.

In [48]:
## Populate CREATE TABLE statment into query variable and execute it.
query = "CREATE TABLE IF NOT EXISTS music_app_history_song"
query = query + "(song text, user_id int, first_name text, last_name text, primary key(song, user_id))"  
try:
    session.execute(query)
except Exception as e:
    print(e)                    

#### Load data into table `music_app_history_song`

In [49]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        ## Populate the INSERT statements into the `query` variable to insert data for song, user_id, first_name and last_name
        query = "INSERT INTO music_app_history_song(song, user_id, first_name, last_name)"
        query = query + "values(%s, %s, %s, %s)"
        ## Execute the INSERT statement by passing the respective column values.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT on `music_app_history_song` table based on following conditions to verify the data.
 * Columns to be selected: user name (first and last) 
 * WHERE Condition : song 'All Hands Against His Own'

In [50]:
# Execute the SELECT statement
query = "select first_name, last_name from music_app_history_song where song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Loop thru the SELECT query results and display the output in console
for row in rows:
    print (row.first_name, '|', row.last_name)

Jacqueline | Lynch
Tegan | Levine
Sara | Johnson


### Drop the tables before closing out the sessions

In [51]:
for table in ['music_app_history_session_item','music_app_history_user_session','music_app_history_song']:
    query = "DROP TABLE IF EXISTS " + table
    try:
        session.execute(query)
        print("Dropped table " + table)
    except Exception as e:
        print(e)

Dropped table music_app_history_session_item
Dropped table music_app_history_user_session
Dropped table music_app_history_song


### Close the session and cluster connection¶

In [52]:
session.shutdown()
cluster.shutdown()